## Installation

In [ ]:
%%writefile requirements.txt
pandas
pandas_datareader
greykite

In [ ]:
%pip install -U --user -r requirements.txt

## Import library

In [ ]:
import warnings
import datetime
import pandas_datareader.data as web
import plotly
import greykite
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries
from greykite.algo.forecast.silverkite.forecast_simple_silverkite_helper import cols_interact
from greykite.algo.forecast.silverkite.constants.silverkite_seasonality import SilverkiteSeasonalityEnum
from greykite.algo.changepoint.adalasso.changepoint_detector import ChangepointDetector
from greykite.algo.forecast.silverkite.constants.silverkite_column import SilverkiteColumn
from greykite.common.features.timeseries_features import get_available_holidays_across_countries
from greykite.framework.input.univariate_time_series import UnivariateTimeSeries
from greykite.framework.templates.autogen.forecast_config import ForecastConfig, ComputationParam, ModelComponentsParam
from greykite.framework.templates.autogen.forecast_config import MetadataParam
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum

## Settings

In [ ]:
%matplotlib inline

In [ ]:
warnings.simplefilter('ignore')

## Fetch dataset

In [ ]:
start = datetime.date(2012,1,1)
end = datetime.date.today()

# TOYOTA(7203.JP)
df = web.DataReader('7203.JP', 'stooq', start, end).sort_index()
df.reset_index(inplace=True)

df.head()

## Analyze timeseries

### Set target col

In [ ]:
time_col = "Date"
value_col = "Close"

### Convert dataframe to greykite timeseries

In [ ]:
ts = UnivariateTimeSeries()
ts.load_data(
    df=df,
    time_col=time_col,
    value_col=value_col,
    freq="D",
)

### Plot original data and moving average

In [ ]:
# original data
fig = ts.plot()
plotly.io.show(fig)

# average per week
fig = ts.plot_grouping_evaluation(
    groupby_sliding_window_size=7,
    title="Weekly average")
plotly.io.show(fig)

# average per month
fig = ts.plot_grouping_evaluation(
    groupby_sliding_window_size=30,
    title="Monthly average")
plotly.io.show(fig)

### Plot quantiles

In [ ]:
# quantiles per week
fig = ts.plot_quantiles_and_overlays(
    groupby_time_feature="str_dow",
    show_mean=True,
    show_quantiles=[],
    center_values=True,
    title="Quantiles per week"
)
plotly.io.show(fig)

# quantiles per year
fig = ts.plot_quantiles_and_overlays(
    groupby_time_feature="month",
    show_mean=True,
    show_quantiles=[],
    center_values=True,
    title="Quantiles per year"
)
plotly.io.show(fig)

### Plot trend and sesonality

In [ ]:
model = ChangepointDetector()

# trend
model.find_trend_changepoints(
    df=df,
    time_col=time_col,
    value_col=value_col,
    yearly_seasonality_order=15,
    regularization_strength=0.1,
    resample_freq="1D",
    potential_changepoint_n=30,
    no_changepoint_distance_from_end="14D",
    actual_changepoint_min_distance="1D",
)

# seasonality
model.find_seasonality_changepoints(
    df=df,
    time_col=time_col,
    value_col=value_col,
)

fig = model.plot(
    observation=True,
    trend_estimate=True,
    adaptive_lasso_estimate=False,
    trend_change=False,
    seasonality_change=False,
    yearly_seasonality_estimate=True,
    seasonality_change_by_component=False,
    seasonality_estimate=True,   
    plot=False)

plotly.io.show(fig)

## Forecast

### Hyper Parameter setting

In [ ]:
metadata = MetadataParam(
    time_col=time_col,
    value_col=value_col,
    freq='D',
)

growth = dict(growth_term="linear")

changepoints = {
    "changepoints_dict": {
        "method": "auto",
        "yearly_seasonality_order": 15,
        "regularization_strength": 0.1,
        "resample_freq": "1D",
        "potential_changepoint_n": 30,
        "no_changepoint_distance_from_end": "14D",
        "actual_changepoint_min_distance": "1D",
    }
}

seasonality = dict(
    yearly_seasonality=True,
    quarterly_seasonality=False,
    monthly_seasonality=False,
    weekly_seasonality=True,
    daily_seasonality=False,
)

autoreg_dict="auto"

model_components = ModelComponentsParam(
    growth=growth,
    changepoints=changepoints,
    seasonality=seasonality,
    autoregression=dict(autoreg_dict=autoreg_dict),
    custom={"fit_algorithm_dict": {"fit_algorithm": "gradient_boosting"}},
)

### Forecast 1 year ahead

In [ ]:
forecaster = Forecaster()
result = forecaster.run_forecast_config(
    df=df,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=365,
        coverage=0.95,
        metadata_param=metadata,
        model_components_param=model_components,
    )
)

plotly.io.show(result.backtest.plot())
print("RMSE :", result.backtest.test_evaluation["RMSE"])

### Forecast 2 year ahead

In [ ]:
forecaster = Forecaster()
result = forecaster.run_forecast_config(
    df=df,
    config=ForecastConfig(
        model_template=ModelTemplateEnum.SILVERKITE.name,
        forecast_horizon=365*2,
        coverage=0.95,
        metadata_param=metadata,
        model_components_param=model_components,
    )
)

plotly.io.show(result.backtest.plot())
print("RMSE :", result.backtest.test_evaluation["RMSE"])